In [22]:
import pandas as pd
import numpy as np
train_data = pd.read_csv('train.csv')

In [23]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [53]:
remove_columns = ['Name','Ticket','Cabin']

X_data = train_data.drop(remove_columns,axis=1)
X_data = train_data.drop('Survived',axis = 1)

In [27]:
Y_data = train_data['Survived']


In [28]:
#Importing the train data split
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_data,Y_data,test_size=0.2)



In [30]:
#We have two object columns!
[col for col in X_train.columns if X_train[col].dtype == 'object']

['Sex', 'Embarked']

In [38]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

categorical_cols = [col for col in train_data.columns if train_data[col].dtype == 'object' and train_data[col].nunique() < 10]
numerical_cols = [col for col in X_train.columns if X_train[col].dtype in ['int64','float64']]

numerical_transformer = SimpleImputer(strategy='mean')

categorical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer,numerical_cols),
        ('cat',categorical_transformer,categorical_cols)
    ]
)


In [91]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor
from xgboost import XGBClassifier


model_1 = RandomForestClassifier(n_estimators=150,max_depth=6,random_state=0)
model_2 = XGBRegressor(n_estimators=500)
model_3 = XGBClassifier(
    objective='binary:logistic',
    n_estimators=300,
    learning_rate=0.1,
    max_depth=3,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    seed=42  # for reproducibility
)



My_pipeline = Pipeline(steps = [
    ('preprocessor',preprocessor),
    ('model',model_1)
])


My_pipeline_XGB = Pipeline(steps = [
    ('preprocessor',preprocessor),
    ('model',model_2)
])

My_pipeline_X_class = Pipeline(steps = [
    ('preprocessor',preprocessor),
    ('model',model_3)
])


In [92]:
My_pipeline.fit(X_train,Y_train)
My_pipeline_XGB.fit(X_train,Y_train)
My_pipeline_X_class.fit(X_train,Y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['PassengerId', 'Pclass',
                                                   'Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked'])])),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callba...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=3, max_leaves=None, min_child_weight=1,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=300,
                               n_jobs=None, num_parallel_tree=None,
                               random_state=None, ...))])

In [93]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(My_pipeline,X_train,Y_train,cv=5,scoring='accuracy') #We can then compare across entire data set!!
print(scores)

[0.79020979 0.81818182 0.80985915 0.78873239 0.83098592]


In [94]:
from sklearn.metrics import accuracy_score
preds = My_pipeline.predict(X_val)
accuracy_score(Y_val,preds)

0.8435754189944135

In [96]:
preds_2 = My_pipeline_X_class.predict(X_val)
accuracy_score(Y_val,preds_2)

0.8100558659217877

Now to use this Pipeline on the test data!

In [97]:
test_data = pd.read_csv('test.csv')

In [98]:
test_X = test_data.drop(remove_columns,axis = 1)


In [99]:
test_predict = My_pipeline_X_class.predict(test_X)

In [100]:
d = {'PassengerId':test_data['PassengerId'],
     'Survived':test_predict}

submission_data = pd.DataFrame(data = d)
submission_data.to_csv('submission.csv',index=False)